In [21]:
import numpy as np 
import pandas as pd 
import matplotlib as plt 
import seaborn as sns 


In [22]:
df = pd.read_csv('Disaster.csv') 
df.head() 

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,index,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,water,food,shelter,clothing,money,missing_people,refugees,death,other_aid,infrastructure_related,transport,buildings,electricity,tools,hospitals,shops,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,0,0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0
2,2,2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,4,4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## First Insight into the Data 

In [41]:
df = df.drop(columns=['Unnamed: 0'])
df.head()

,index,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,water,food,shelter,clothing,money,missing_people,refugees,death,other_aid,infrastructure_related,transport,buildings,electricity,tools,hospitals,shops,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0
2,2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [23]:
df.columns 

Index(['Unnamed: 0', 'index', 'id', 'message', 'original', 'genre', 'related',
       'request', 'offer', 'aid_related', 'medical_help', 'medical_products',
       'search_and_rescue', 'security', 'military', 'child_alone', 'water',
       'food', 'shelter', 'clothing', 'money', 'missing_people', 'refugees',
       'death', 'other_aid', 'infrastructure_related', 'transport',
       'buildings', 'electricity', 'tools', 'hospitals', 'shops',
       'aid_centers', 'other_infrastructure', 'weather_related', 'floods',
       'storm', 'fire', 'earthquake', 'cold', 'other_weather',
       'direct_report'],
      dtype='object')

In [24]:
df.isnull().sum()

Unnamed: 0                    0
index                         0
id                            0
message                       0
original                  16046
genre                         0
related                       0
request                       0
offer                         0
aid_related                   0
medical_help                  0
medical_products              0
search_and_rescue             0
security                      0
military                      0
child_alone                   0
water                         0
food                          0
shelter                       0
clothing                      0
money                         0
missing_people                0
refugees                      0
death                         0
other_aid                     0
infrastructure_related        0
transport                     0
buildings                     0
electricity                   0
tools                         0
hospitals                     0
shops   

In [25]:
df.genre.unique()

array(['direct', 'social', 'news'], dtype=object)

## Data Cleaning

In [26]:
from nltk import download 

download('punkt')
download('stopwords')
download('averaged_perceptron_tagger')
download('maxent_ne_chunker')
download('words')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [27]:
#Tokenizaton

from nltk.tokenize import word_tokenize 
from nltk.stem import PorterStemmer 
from nltk.corpus import stopwords 

def create_tokens(df, category): 
    tokenized = []
    for i in df[df[category] == 1].index:
        text = df.loc[i, 'message']
        tokens = word_tokenize(text)
        tokenized.extend([[token for token in tokens if token.isalnum()]]) # Remove punctuations 

    for tokens in tokenized:   # Removing stopwords 
        for token in tokens:
            if token in stopwords.words('english'):
                tokens.remove(token)  
    
    return tokenized 


In [28]:
# POS Tagging 

from nltk import pos_tag 

def create_pos(tokenized_list):
    pos_list = []
    for i in range(len(tokenized_list)):      
        pos_list.extend([pos_tag(tokenized_list[i])]) 
    
    return pos_list 


In [29]:
# Named Entity Recognition

from nltk import ne_chunk, Tree 

def get_locations(pos_words): 
    locations = []
    for i in range(len(pos_words)): 
        chunks = ne_chunk(pos_words[i]) 
        for c in chunks: 
            if isinstance(c, Tree) and c.label() == 'GPE': 
                # The object is <class 'nltk.tree.Tree'> and label is Geopolitical Entity
                locations.extend([w for w,_ in c.leaves()])
    
    return locations

In [30]:
# Testing for weather disaster category 

tokenized = create_tokens(df, 'weather_related') 
pos_words = create_pos(tokenized) 
locations_weather = get_locations(pos_words) 

In [37]:
from collections import Counter

wrong_words = ['Good','Please','Heavy'] 
for word in wrong_words: 
  locations_weather = ' '.join(locations_weather).replace(word, '').split()

Counter(locations_weather).most_common(25)

[('China', 146),
 ('Pakistan', 137),
 ('India', 95),
 ('Indonesia', 80),
 ('New', 75),
 ('Indian', 68),
 ('Haiti', 58),
 ('South', 41),
 ('Indonesian', 40),
 ('York', 32),
 ('Afghanistan', 30),
 ('Japan', 29),
 ('Taiwan', 27),
 ('Chinese', 26),
 ('Thailand', 25),
 ('Asia', 22),
 ('Santiago', 20),
 ('North', 19),
 ('Haitian', 19),
 ('United', 19),
 ('BEIJING', 19),
 ('Sandy', 18),
 ('Chile', 17),
 ('States', 17),
 ('Russia', 16)]

## Graphical Visualization

In [42]:
import matplotlib as plt  
